# Depreciation Flags

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# Standard libraries
import logging
import sys
import os
import pathlib

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

In [2]:
# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [3]:
# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 5

In [4]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq='AS'
)

In [11]:
fbp = pudl_out.fbp_ferc1()
#fbp['primary_fuel_by_mmbtu']
#fbp['primary_fuel_by_cost']

In [12]:
steam = pudl_out.plants_steam_ferc1()
steam
# Previously was 27470 rows

,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor,capacity_mw,capex_equipment,capex_land,capex_per_mw,capex_structures,capex_total,construction_type,construction_year,installation_year,net_generation_mwh,not_water_limited_capacity_mw,opex_allowances,opex_boiler,opex_coolants,opex_electric,opex_engineering,opex_fuel,opex_fuel_per_mwh,opex_misc_power,opex_misc_steam,opex_nonfuel_per_mwh,opex_operations,opex_per_mwh,opex_plants,opex_production_total,opex_rents,opex_steam,opex_steam_other,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw
0,1994,1,7,AEP Generating Company,530,1056,rockport unit 1,NaN,NaN,0.819843,650.0,490684127.0,6395551.0,894688.3,84467746.0,581547424.0,conventional,1984.0,1984.0,4668184.0,650.0,NaN,3185935.0,NaN,353599.0,427906.0,51694529.0,11.073799,1040610.0,781181.0,1.778100,1032559.0,12.9,631598.0,59995027.0,7559.0,442763.0,NaN,396788.0,NaN,650.0,NaN,NaN,steam,f1_steam_1994_12_1_0_1,NaN
1,1994,1,7,AEP Generating Company,530,1057,rockport unit 2,NaN,NaN,0.781755,650.0,39339370.0,74411.0,67173.7,4249136.0,43662917.0,conventional,1989.0,1989.0,4451312.0,650.0,NaN,3374827.0,NaN,384283.0,427747.0,48990225.0,11.005794,1028788.0,255391.0,16.850051,1026248.0,27.9,518870.0,123995060.0,67311927.0,446454.0,NaN,230300.0,NaN,650.0,NaN,NaN,steam,f1_steam_1994_12_1_0_2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27468,2019,531,582,basin electric power cooperative (pudl determi...,15391,1898,wisdom unit 2 genera,NaN,5.0,0.007125,127.0,15382648.0,NaN,144316.8,2945580.0,18328228.0,unknown,2004.0,2004.0,7927.0,127.0,NaN,NaN,NaN,381424.0,40980.0,312093.0,39.370884,NaN,NaN,89.606913,49593.0,129.0,215320.0,1022407.0,NaN,NaN,NaN,22997.0,NaN,83.0,NaN,352.0,combustion_turbine,f1_steam_2019_12_531_2_2,127.0
27469,2019,22,69,Cleco Power LLC,14365,1899,st mary's,NaN,4.0,0.190025,50.9,NaN,NaN,0.0,NaN,NaN,outdoor,2019.0,2019.0,84729.0,47.0,NaN,30137.0,NaN,114333.0,281166.0,NaN,NaN,635890.0,22662.0,NaN,46872.0,20.1,117067.0,1700933.0,NaN,452057.0,NaN,749.0,NaN,47.0,NaN,2896.0,steam,f1_steam_2019_12_22_1_5,NaN


### SQL Access

In [6]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [7]:
plants = pd.read_sql("plants_eia860", pudl_engine)

### Output Table Access

In [8]:
pudl_out.plants_eia860()

,plant_id_eia,plant_name_eia,balancing_authority_code_eia,balancing_authority_name_eia,city,county,ferc_cogen_status,ferc_exempt_wholesale_generator,ferc_small_power_producer,grid_voltage_kv,grid_voltage_2_kv,grid_voltage_3_kv,iso_rto_code,latitude,longitude,primary_purpose_naics_id,sector_name,sector_id,service_area,state,street_address,zip_code,timezone,report_date,ash_impoundment,ash_impoundment_lined,ash_impoundment_status,datum,energy_storage,ferc_cogen_docket_no,ferc_exempt_wholesale_generator_docket_no,ferc_small_power_producer_docket_no,liquefied_natural_gas_storage,natural_gas_local_distribution_company,natural_gas_storage,natural_gas_pipeline_name_1,natural_gas_pipeline_name_2,natural_gas_pipeline_name_3,nerc_region,net_metering,pipeline_notes,regulatory_status_code,transmission_distribution_owner_id,transmission_distribution_owner_name,transmission_distribution_owner_state,utility_id_eia,water_source,plant_id_pudl,utility_name_eia,utility_id_pudl
0,1,Sand Point,None,None,Sand Point,Aleutians East,False,False,False,0.48,NaN,NaN,None,55.339722,-160.497222,22.0,IPP Non-CHP,2.0,None,AK,100 Power Plant Way,99661,America/Anchorage,2019-01-01,False,False,None,None,False,None,None,None,False,None,False,None,None,None,None,None,None,NR,63560,"TDX Sand Point Generating, LLC",AK,63560,None,15346,"TDX Sand Point Generating, LLC",6409
1,2,Bankhead Dam,SOCO,"Southern Company Services, Inc. - Trans",Northport,Tuscaloosa,False,False,False,115.00,NaN,NaN,None,33.458665,-87.356823,22.0,Electric Utility,1.0,None,AL,19001 Lock 17 Road,35476,America/Chicago,2019-01-01,False,False,None,None,False,None,None,None,None,None,None,None,None,None,SERC,None,None,RE,195,Alabama Power Co,AL,195,Black Warrior River,854,Alabama Power Co,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127828,63940,Central 40,CISO,California Independent System Operator,Newman,Stanislaus,False,False,False,115.00,NaN,NaN,None,37.246000,-121.104000,22.0,IPP Non-CHP,2.0,None,CA,4951 Sullivan Road,95360,America/Los_Angeles,2019-01-01,None,None,None,None,False,None,None,None,None,None,None,None,None,None,WECC,None,None,NR,14328,Pacific Gas & Electric Co.,CA,63606,None,14628,NaN,<NA>
127829,63941,Oasis Alta,CISO,California Independent System Operator,Mojave,Kern,False,False,False,230.00,NaN,NaN,None,35.059166,-118.291944,22.0,IPP Non-CHP,2.0,None,CA,TBD,93501,America/Los_Angeles,2019-01-01,None,None,None,None,False,None,None,None,None,None,None,None,None,None,WECC,None,None,NR,17609,Southern California Edison Co,CA,63477,None,14690,NaN,<NA>
